In [29]:
import pandas as pd
import os
import sys
import json
import re

In [72]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )

# Define data output path
transcript_path = os.path.join(git_dir, 'data', 'pm-transcripts-processed/aggregated-spellfixed.csv')
df = pd.read_csv(transcript_path)

In [73]:
df['content'] = df['content'].str.replace(u'\xa0', u' ')
df['content'] = df['content'].str.replace(' \]\]\>', '')

In [74]:
df

,Unnamed: 0,pm,date,content,sentences
0,0,"Abbott, Tony",24/08/2014,Every day the Government is working to build a...,['every day the government is working to build...
1,1,"Gorton, John",14/03/1968,"STATEMENT BY THE PRIME MINISTER, THE RT. HON....",['the house of representatives on a of march o...
2,2,"Turnbull, Malcolm",17/11/2016,PRIME MINISTER: This is a great initiative. Wh...,['what you re seeing here is real enthusiasm f...
3,3,"Fraser, Malcolm",27/09/1978,"FOR PRESS 27 SEPTEMBER, 1978 GRANT TO WORLD W...",['for press of september of of grant to world ...
4,4,"Howard, John",24/03/2004,"JOURNALIST: Prime Minister, is Mark Latham pla...","[""journalist prime minister is mark latham pla..."
...,...,...,...,...,...
22804,22804,"Turnbull, Malcolm",13/11/2016,The Australian Government has reached a furthe...,['the australian government has reached a furt...
22805,22805,"Gorton, John",20/09/1970,FOR PRESS: P. M. No 92-1970 CHISHOLM BY-ELECT...,['no of of of chisholm by election result comm...
22806,22806,"Howard, John",08/09/2006,I was saddened today to learn of the death of ...,['i was saddened today to learn of the death o...
22807,22807,"Abbott, Tony",29/11/2014,Today is a landmark day in the fight against d...,['today is a landmark day in the fight against...
